# Logic Algorithms
First realize some basic discriminance.

In [50]:
from sympy import symbols, Function, Implies
from sympy.core.function import AppliedUndef
from sympy.core.symbol import Symbol
from sympy.core.expr import Expr

class Substitution(dict):
    '''
    Use a dictionary to represent a substitution.
    '''
    def __str__(self):
        return '{'+', '.join(f'{k}/{v}' for k, v in self.items())+'}'

def is_expr(x):
    return x.is_symbol or isinstance(x, AppliedUndef)

def is_var(x):
    return x.is_symbol and str(x).islower()

def is_const(x):
    return x.is_symbol and not str(x).islower()

def is_comp(x):
    return isinstance(x, AppliedUndef)

def is_op(x):
    return isinstance(x, Function)

class KBase():
    def __init__(self) -> None:
        self._rules = set()
        self._facts = set()
    def __str__(self) -> str:
        rules = '\n'.join(str(r) for r in self._rules)
        facts = '\n'.join(str(f) for f in self._facts)
        return f'Rules:\n{rules}\nFacts:\n{facts}'
    def tell(self, t:Expr) -> None:
        if isinstance(t, Implies):
            self._rules.add(t)
        else:
            self._facts.add(t)
        

### Unification Algorithm
Try to substitute variables to unify two expressions, making them the same.

In [51]:
def unify(expr1:Expr, expr2:Expr, theta:Substitution=None)->Substitution:
    if theta is None: # exsiting a failure in recurrence
        theta = {}
    if expr1 == expr2: # already the same
        return theta
    elif is_var(expr1): 
        return unify_var(expr1, expr2, theta)
    elif is_var(expr2):
        return unify_var(expr2, expr1, theta)
    elif is_comp(expr1) and is_comp(expr2):
        if expr1.func == expr2.func and len(expr1.args) == len(expr2.args):
            for arg1, arg2 in zip(expr1.args, expr2.args):
                theta = unify(arg1, arg2, theta)
                if theta is None:
                    return None
            return theta
    return None

def unify_var(var: Symbol, expr:Expr, theta)->Substitution:
    '''
    Unify a variable with an expression.
    '''
    if var in theta:
        return unify(theta[var], expr, theta)
    elif expr in theta:
        return unify(var, theta[expr], theta)
    elif occur_check(var, expr, theta):
        return None
    else:
        theta[var] = expr
        return theta

def occur_check(var:Symbol, expr:Expr, theta)->bool:
    '''
    Check if a variable occurs in a term.
    '''
    if var == expr:
        return True
    elif is_comp(expr):
        return any(occur_check(var, arg, theta) for arg in expr.args)
    return False

Here we have some test examples.

In [52]:
x, y = symbols('x y')
John, Jane, Bill, Elizabeth = symbols('John Jane Bill Elizabeth')
Mother = Function('Mother')
Knows = Function('Knows')

expr1 = Knows(John, x)
expr2 = Knows(John, Jane)
expr3 = Knows(y, Bill)
expr4 = Knows(y, Mother(y))
expr5 = Knows(y, Elizabeth)

print("UNIFY(Knows(John, x), Knows(John, Jane)) =", unify(expr1, expr2))
print("UNIFY(Knows(John, x), Knows(y, Bill)) =", unify(expr1, expr3))
print("UNIFY(Knows(John, x), Knows(y, Mother(y))) =", unify(expr1, expr4))
print("UNIFY(Knows(John, x), Knows(y, Elizabeth)) =", unify(expr1, expr5))

UNIFY(Knows(John, x), Knows(John, Jane)) = {x: Jane}
UNIFY(Knows(John, x), Knows(y, Bill)) = {y: John, x: Bill}
UNIFY(Knows(John, x), Knows(y, Mother(y))) = {y: John, x: Mother(y)}
UNIFY(Knows(John, x), Knows(y, Elizabeth)) = {y: John, x: Elizabeth}


## Forward Chaining
Beginning from rules and knowledge in the Knowledge Base, infer forward to get many new knowleadge. Maybe one of the new ones is what we need.

In [53]:
from itertools import product

def fc_fool(self, query:Expr)->bool:
    '''
    Forward chaining inference, to match a query.
    '''
    if any(unify(fact, query) is not None for fact in self._facts):
        return True
    while True:
        new = set()
        for rule in self._rules:
            premises = rule.args[0]
            conclusion = rule.args[1]
            for fact_tuple in product(self._facts, repeat=len(premises.args)-1):
                subst = Substitution()
                for fact, premise in zip(fact_tuple, premises.args):
                    theta = unify(fact, premise, subst)
                    if theta is None:
                        break
                    subst |= theta
                else:
                    new_fact = conclusion.subs(subst)
                    if new_fact not in self._facts:
                        new.add(new_fact)
        if not new:
            return False
        self._facts |= new
        print(new)
        if any(unify(fact, query) is not None for fact in self._facts):
            return True

KBase.fc_fool = fc_fool

Here are some test samples.

In [54]:
from sympy import Function, And, Implies
from sympy.abc import x, y, z, a, b, c

kb = KBase()
American, Weapon, Sells, Hostile, Criminal, Owns, Missle, Enemy = \
    Function('American', bool=True), Function('Weapon', bool=True), Function('Sells', bool=True), \
    Function('Hostile', bool=True), Function('Criminal', bool=True), Function('Owns', bool=True), \
    Function('Missle', bool=True), Function('Enemy', bool=True)
And, Or, Not = Function('And'), Function('Or'), Function('Not')
Nono, West, America, M1 = symbols('Nono West America M1')
# rules
kb.tell(Implies(And(American(x), Weapon(y), Sells(x, y, z), Hostile(z)), Criminal(x)))
kb.tell(Implies(And(Missle(a), Owns(Nono, a)), Sells(West, a, Nono)))
kb.tell(Implies(And(Missle(b)), Weapon(b)))
kb.tell(Implies(And(Enemy(c, America)), Hostile(c)))
# facts

kb.tell(Owns(Nono, Missle(M1)))
kb.tell(American(West))
kb.tell(Missle(M1))
kb.tell(Enemy(Nono, America))

kb.fc_fool(Criminal(West)) # True

{Weapon(b), Hostile(c), Sells(West, M1, Nono)}
{Criminal(West)}


True

## Backward Chaining
Beginning from query, 

In [55]:
def bc_fool(self, query:Expr)->bool:
    '''
    Backward chaining inference, to match a query.
    '''
    if any(unify(fact, query) is not None for fact in self._facts):
        return True
    for rule in self._rules:
        premises = rule.args[0]
        conclusion = rule.args[1]
        subst = Substitution()
        for premise in premises.args:
            theta = unify(premise, query, subst)
            if theta is not None:
                break
        else:
            continue
        new_query = conclusion.subs(theta)
        if self.bc_fool(new_query):
            return True
    return False

KBase.bc_fool = bc_fool

In [56]:
kb.bc_fool(Criminal(West)) # True

True

## Resolution

In [60]:
def resolve(self, query:Expr)->bool:
    '''
    Resolve a query.
    '''
    return self.fc_fool(query) or self.bc_fool(query)

def DNF(expr:Expr):
    '''
    Convert an expression to disjunctive normal form.
    '''
    if isinstance(expr, Implies):
        return DNF(Or((Not(expr.args[0]), expr.args[1])))
    elif isinstance(expr, And):
        return Or(*[Not(DNF(arg)) for arg in expr.args])
    elif isinstance(expr, AppliedUndef):
        return expr
    return expr

def to_DNF(self):
    '''
    Convert all rules and facts into DNF.
    '''
    self._rules = {DNF(rule) for rule in self._rules}
    self._facts = {DNF(fact) for fact in self._facts}

KBase.resolve = resolve
KBase.to_DNF = to_DNF

In [61]:
print(DNF(Implies(And(American(x), Weapon(y), Sells(x, y, z), Hostile(z)), Criminal(x))))
kb.to_DNF()
print(kb)

Or((Not(And(American(x), Weapon(y), Sells(x, y, z), Hostile(z))), Criminal(x)))
Rules:
Or((Not(And(American(x), Weapon(y), Sells(x, y, z), Hostile(z))), Criminal(x)))
Or((Not(And(Enemy(c, America))), Hostile(c)))
Or((Not(And(Missle(b))), Weapon(b)))
Or((Not(And(Missle(a), Owns(Nono, a))), Sells(West, a, Nono)))
Facts:
Criminal(West)
Enemy(Nono, America)
Owns(Nono, Missle(M1))
American(West)
Weapon(b)
Hostile(c)
Missle(M1)
Sells(West, M1, Nono)
